In [13]:
import torch
import os
import numpy as np
import torch.nn.functional as F
from helper.model import WordLSTM
import random
import sys
from helper.model import WordLSTM

In [29]:
sys.path.append('/home/taindp/VINBRAIN_INTERNSHIP/nlg_lstm/code/helper')

In [30]:
%pwd

'/home/taindp/VINBRAIN_INTERNSHIP/nlg_lstm/code'

In [31]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath,map_location=lambda storage, loc: storage)
    model = checkpoint['model']
    model.load_state_dict(checkpoint['state_dict'])
    for parameter in model.parameters():
        parameter.requires_grad = False

    model.eval()
    return model

In [32]:
# net = WordLSTM()

In [33]:
_MODEL_PATH = '/home/taindp/VINBRAIN_INTERNSHIP/nlg_lstm/model'
_RESOURCE_PATH = '/home/taindp/VINBRAIN_INTERNSHIP/nlg_lstm/resource'

In [50]:
int2token = torch.load(os.path.join(_RESOURCE_PATH,'int2token_vi.h5'))

token2int = torch.load(os.path.join(_RESOURCE_PATH,'token2int_vi.h5'))

vocab_size = len(int2token)

net = WordLSTM(vocab_size)

# print(net)

In [51]:
net

WordLSTM(
  (emb_layer): Embedding(656, 200)
  (lstm): LSTM(200, 256, num_layers=4, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=656, bias=True)
)

In [46]:
torch.__version__

'1.7.0+cpu'

In [52]:
net = load_checkpoint(os.path.join(_MODEL_PATH,'checkpoint_vi.pth'))

AttributeError: Can't get attribute 'WordLSTM' on <module 'model' (namespace)>

In [48]:
net

WordLSTM(
  (emb_layer): Embedding(16592, 200)
  (lstm): LSTM(200, 256, num_layers=4, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=16592, bias=True)
)

In [ ]:
def predict(net,tkn,h=None):
    
    x = np.array([[token2int[tkn]]])
    inputs = torch.from_numpy(x)
    
#     inputs = inputs.cuda()
    h = tuple([each.data for each in h])
    
    out,h = net(inputs,h)
    
    p = F.softmax(out,dim=1).data
    
#     p = p.cuda()
    p = p.numpy()
    p = p.reshape(p.shape[1],)
    
    top_n_idx = p.argsort()[-3:][::-1]
    
    sampled_token_index = top_n_idx[random.sample([0,1,2],1)[0]]
    
    return int2token[sampled_token_index],h
    

In [ ]:
def sample(net,size,prime='it is'):
#     net.cuda()
    net.eval()
    
    h = net.init_hidden(1)
    toks = prime.split()
    
    #predict next token
    
    for t in prime.split():
        token,h = predict(net,t,h)
        
    toks.append(token)
    
    for i in range(size-1):
        token,h = predict(net,toks[-1],h)
        toks.append(token)
    return ' '.join(toks)

In [ ]:
sample(net,15,prime = 'one of the')

In [ ]:
sample(net,5,prime = 'one of the')